<a href="https://colab.research.google.com/github/nezihaksu/Airbnb_Istanbul/blob/main/airbnb_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from sklearn.impute import SimpleImputer
import re

#Explatory Data Analysis

In [ ]:
DF = r'/content/drive/MyDrive/listings.csv'
FILE_TYPE = "csv"
IMPUTE = True
ALLOWED_NAN_PERCENTAGE = 10
DROP_KEYWORDS = ["code","zipcode","link","url","id","name","thumbnail","picture","pic","description","note"]

class Eda():
  """Explore the dataset."""
  def __init__(self,df,file_type:str):
    if file_type == "xlsx" or  file_type == "xls":
      self.df = pd.read_excel(df,engine="python")
    self.df = pd.read_csv(df,engine="python")
    self.file_type = file_type

  def __call__(self):
    return self.df

  def intro(self):
    return "===INFO===",self.df.info(),"===DESCRIPTION===",self.df.describe(),"===DTYPES==",self.df.dtypes
  
  def unique_values(self):
    #Unique values that are in features.
    for column in self.df.columns:
      print(column.upper()+ " UNIQUE VALUES")
      print(str(df[column].unique())+"\n")

  def missing_values(self):
	  missing_percentage = self.df.isnull().sum()*100/len(self.df)
	  plt.figure(figsize=(5, 15))
	  missing_percentage.plot(kind='barh')
	  plt.xticks(rotation=90, fontsize=10)
	  plt.yticks(fontsize=5)
	  plt.xlabel("Missing Percentage", fontsize=14)
	  plt.show()
   
  #Plotting histograms of the numerical features to see the distribution of each of them.
  def dtype_histogram(self,data_type:str):
    numerical_features = self.df.dtypes[self.df.dtypes == data_type].index.to_list()
    self.df[numerical_features].hist(bins = 50,figsize = (20,15))
    plt.show()

In [ ]:
eda = Eda(df=DF,file_type=FILE_TYPE)

In [ ]:
class Preprocess():
  """Clean the dataset."""
  def __init__(self,df):
    self.df = df

  #Expanding one column dataframe into multiple columns according to split character.
  def split_column_into_df(self,column_index:int,split_char:str):
    if len(df.columns) == 1:
      quotes_strip = list(self.df.columns)[0].replace(strip_char,'')
      columns_split = quotes_strip.split(split_char)
      self.df = self.df[self.df.iloc[:,0].name].str.split(pat = split_char,expand = True)
      self.df.columns =  columns_split
      self.df.replace(split_char,'',regex = True,inplace = True)
    print("This method is only for explanding single column dataframes!")
    return self.df

  def drop_missing_columns(self,percentage):
    missing_percentage = self.df.isnull().sum()*100/len(self.df)
    features_left = missing_percentage[missing_percentage < percentage].index
    return self.df[features_left]

  def drop_column_contains(self,keywords:list):
    for keyword in keywords:
      keyword_pattern = re.compile(keyword)
      for column in self.df.columns:
        if keyword_pattern.search(column):
          self.df.drop(column,axis=1,inplace=True)
    return self.df

  #Loops over the values of the column until hitting the specified limit to find letter character to match and drop the column.
  def _drop_value_type(self,pattern,search_limit:int):
      for column in self.df.columns:
        for value in self.df[column].values[:search_limit]:
          if pattern.match(str(value)):
            print(column)
            self.df.drop(column,axis=1,inplace=True)

      return self.df

  def drop_text_columns(self,search_limit:int):
    letter_pattern = re.compile(r'[A-z]')

    return self._drop_value_type(letter_pattern,search_limit)
    
  def drop_date_columns(self,search_limit:int):
    date_delimeters = ["/","-","."," "]
    for delimeter in date_delimeters:
      date_pattern = re.compile(r'([12]\d{3}{}(0[1-9]|1[0-2]){}(0[1-9]|[12]\d|3[01]))'.format(delimeter))

      return _drop_value_type(date_pattern,search_limit)


In [ ]:
df = eda()
preprocess = Preprocess(df)